# NN from scratch with Titanic data set

In [ ]:
import torch, numpy as np, pandas as pd
df = pd.read_csv("train.csv")
df.isna().sum()

In [ ]:
modes = df.mode().iloc[0]
modes

In [ ]:
df.fillna(modes, inplace=True)
df.isna().sum()

In [ ]:
# log of fare 
df['LogFare'] = np.log(df['Fare']+1)

In [ ]:
df = pd.get_dummies(df, columns=["Sex","Pclass","Embarked"])
df.columns

In [ ]:
from torch import tensor

# dependent variable Y is survived 
t_dep = tensor(df["Survived"])

# independent variably X are the rest (exclude str vars: name, ticket, cabin)
cols = ['Age', 'SibSp', 'Parch', 'LogFare', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S']

t_indep = tensor(df[cols].astype(float).values, dtype=torch.float)
t_indep

In [ ]:
df[["Embarked_C","Embarked_Q"]].astype(int)

df[cols].head()

## Linear Model with OLS

Remember: 
$
(X'X)^{-1} X'y
$

In [ ]:
ones = np.ones( (len(t_indep),1) )

cols = ['Age', 'SibSp', 'Parch', 'LogFare', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Embarked_C', 'Embarked_Q']

t_indep_lin = tensor(df[cols].astype(float).values, dtype=torch.float)


X = np.array(t_indep_lin)/np.array(t_indep_lin).max(axis=0) 
#X = np.vstack( (ones,X) ) #[np.ones(len(t_indep))
X_ = np.append( ones ,X ,axis=1)
y = np.array(t_dep)

ols_coeffs = np.linalg.inv(X_.T.dot(X_)).dot(X_.T.dot(y))
ols_coeffs = dict(zip( ["const"]+cols , ols_coeffs))
ols_coeffs

In [ ]:
from sklearn import linear_model
import statsmodels.api as sm

reg = linear_model.LinearRegression()
reg.fit(X,y)
#LinearRegression()
print(len(reg.coef_))
sk_coeffs = dict(zip( cols , reg.coef_))
sk_coeffs

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#import seaborn as sbs
#sbs.pairplot(df[cols])

In [ ]:
# normalizing the data to 0-1
t_indep = t_indep/t_indep.max(dim=0).values
t_indep

## Setting up the NN 

In [ ]:
torch.manual_seed(442)

n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

In [ ]:
preds = (t_indep*coeffs).sum(axis=1)
loss = torch.abs(preds-t_dep).mean()
loss

In [ ]:
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [ ]:
coeffs.requires_grad_()

In [ ]:
loss = calc_loss(coeffs, t_indep, t_dep)
loss

In [ ]:
loss = calc_loss(coeffs, t_indep, t_dep)
loss.backward()
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    coeffs.grad.zero_()
    print(calc_loss(coeffs, t_indep, t_dep))

## training the model

In [ ]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(df)

trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
len(trn_indep),len(val_indep)

In [ ]:
def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [ ]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [ ]:
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()

In [ ]:
def train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [ ]:
coeffs = train_model(18, lr=0.2)

In [ ]:
def show_coeffs(): return dict(zip(cols, coeffs.requires_grad_(False)))
show_coeffs()

In [ ]:
sk_coeffs